<a href="https://colab.research.google.com/github/rebeccahe1998/ScrapeingStuff/blob/main/PUBLIC_THEMES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import numpy as np
import pandas as pd

In [ ]:
bs4.version

NameError: ignored

# This Notebook finds stocks and tickers for different themes and its description from Public.com
No input is needed, just run all the cells and a Excel file with each sheets correponds to each theme is generated.

In [ ]:
# get every themes's urls
response = requests.get('https://public.com/themes', proxies={'http':'186.248.170.82'})
soup = BeautifulSoup(response.text,'lxml')
url_ =[]
for link in soup.findAll('a',href=True):
    url_.append(link.get('href'))
import re
url_list = []
for i in url_:
    if re.match(r'/themes',i):
        url_list.append('https://public.com'+i)

# get url stock and ticker
def get_df(page_url):
    
    storea = []
    storeb = []
    response = requests.get(page_url)
    if not response.status_code == 200:    # if cannot scrap data from the url
        return None
    
    try:
        soup = BeautifulSoup(response.text,'lxml')
        for a in soup.find_all('h3'):
            storea.append(a.text)
        for b in soup.find_all('p',class_='css-uqwykd'):
            storeb.append(b.text)

        return storea,storeb
    
    except:
        return None


In [ ]:
# get description of each theme
response = requests.get('https://public.com/themes')
soup = BeautifulSoup(response.text,'lxml')
description = []
for a in soup.find_all('p',{'class':'css-ayr41v'}):
    description.append(a.text)


In [ ]:
# save
writer = pd.ExcelWriter('PUBLIC_STOCK_TICKER.xlsx')
for num,i in enumerate(url_list):
    a,b = get_df(i)
    df=pd.DataFrame({'Stock':a,'Ticker:':b} )
    df.loc[0,['Description'] ]= description[num]
    df.to_excel(writer, sheet_name=url_list[num].split('/')[-1])
writer.save()